## Determinar quais provas e quais passos vao ser usados para teste

In [1]:
from tree_parser import file_contents, meta_math_database

from my_utils import print_proof_props_graph, get_proof_steps, print_ident_proof, print_proof_linear_steps
from my_utils import get_proof_steps_graph, print_proof_steps_graph


from expanding import construct_proof, expand_proof_step_ps

from collections import defaultdict

import glob

import os

import numpy as np

from sklearn.metrics import roc_auc_score

import random

from tqdm import tqdm

from expanding import PStep, construct_proof
from theorem_database import TheoremDatabase

In [2]:
tdb = TheoremDatabase("tdb")

In [3]:
%%time

text = file_contents("set_mod.mm")
#database = meta_math_database(text,n=16080)
database = meta_math_database(text,n=5000)

included 3651825 tokens from set_mod.mm
proposition: 5000CPU times: user 22.8 s, sys: 426 ms, total: 23.2 s
Wall time: 23.3 s


In [4]:
def get_theorem_complexity(theorem_name):
    theorem = tdb[theorem_name]
    
    assert theorem != None, f"Error while getting theorem '{theorem_name}'."
    
    if "complexity" not in theorem:
        t_name = theorem["theorem"]
        print(f"Complexity of {t_name} not found. Getting complexity...")
        
        if len(theorem["steps"]) == 0:
            theorem["complexity"] = 1
        else:
            theorem["complexity"] = sum([get_theorem_complexity(tt["theorem"]) for tt in theorem["steps"]])
            
    return theorem["complexity"]

In [5]:
get_theorem_complexity("bitr4i")

10727

### Intensity
Intensity measures how much a formula summarizes information from the leaf ancestors in its derivation tree. Formulae that summarize a lot of information are more interesting. 
The plurality score of a formula (or set of formulae) is number of function and predicate symbols in the formula divided by the number of distinct function and predicate symbols in the formula. Plurality measures the extent to which symbols are repeated in the formula. The intensity score of a formula is the plurality of its leaf ancestors divided by the plurality of the formula itself. A higher score is better. Intensity works well with complexity – interesting things are often compact,
which means intense but not complex.

In [6]:
def get_plurality(statement, no_parenthesis=False):
    exc_symbols = ["(", ")"] if no_parenthesis else []
    return count_tokens(statement, exc_symbols) / count_tokens(statement, exc_symbols, distinct=True) 

def count_tokens(statement, excluded_tokens=[], distinct=False):
    tokens = [t for t in statement.split(" ") if t not in excluded_tokens]
    if distinct:
        count = len(set(tokens))
    else:
        count = len(tokens)
    return count

def get_intensity(step_statement, prev_statements, no_parenthesis=False):
    #In case prev_statements is empty, it will still work because count_tokens will return 1
    #due to empty tokens list and hence plurality will return 1/1 = 1.
    
    #leaves_plur = sum([get_intensity(s) for s in prev_steps])
    leaves_plur = get_plurality(" ".join([s for s in prev_statements]), no_parenthesis)
    form_plur = get_plurality(step_statement, no_parenthesis)
    return leaves_plur / form_plur

prop_proof = construct_proof(database.propositions["dfss2"])
print(get_plurality(prop_proof.statement))
print(get_intensity(prop_proof.statement, [s.statement for s in prop_proof.inputs]))

print(get_plurality(prop_proof.statement, True))
print(get_intensity(prop_proof.statement, [s.statement for s in prop_proof.inputs], True))

1.7
2.8877005347593583
1.625
2.735042735042735


In [7]:
get_plurality(prop_proof.statement)

1.7

In [8]:
prop_proof.statement

'( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )'

In [9]:
" ".join([s.statement for s in prop_proof.inputs])

'( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ) ( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )'

In [10]:
get_plurality(" ".join([s.statement for s in prop_proof.inputs]))

4.909090909090909

In [12]:
def print_linear(stp, ident=0):
    int_value = get_intensity(stp.statement, [s.statement for s in stp.inputs], True)
    print(" | " * ident, stp, int_value)
    for cs in stp.inputs:
        print_linear(cs, ident+1)
        
prop_proof = construct_proof(database.propositions["dfss2"])
prop_proof = prop_proof.expand().get_root_step()
prop_proof = prop_proof.expand().get_root_step()
prop_proof = prop_proof.expand().get_root_step()
print_linear(prop_proof)

 <PStep:mp2 ⊢ ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )> 5.153846153846154
 |  <PStep:sylib ⊢ ( A ⊆ B → ∀ x ( x ∈ A → x ∈ B ) )> 2.393162393162393
 |  |  <PStep:biimpi ⊢ ( A ⊆ B → ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )> 1.125
 |  |  |  <PStep:3bitri ⊢ ( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )> 1.990950226244344
 |  |  |  |  <PStep:dfss ⊢ ( A ⊆ B ↔ A = ( A ∩ B ) )> 0.6666666666666666
 |  |  |  |  <PStep:eqeq2i ⊢ ( A = ( A ∩ B ) ↔ A = { x | ( x ∈ A ∧ x ∈ B ) } )> 0.868421052631579
 |  |  |  |  |  <PStep:df-in ⊢ ( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }> 0.6666666666666666
 |  |  |  |  <PStep:abeq2 ⊢ ( A = { x | ( x ∈ A ∧ x ∈ B ) } ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )> 0.4074074074074074
 |  |  <PStep:bicomi ⊢ ( ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ↔ ∀ x ( x ∈ A → x ∈ B ) )> 1.0
 |  |  |  <PStep:albii ⊢ ( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )> 0.9440993788819876
 |  |  |  |  <PStep:pm4.71 ⊢ ( ( x ∈ A → x ∈ B ) ↔ ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )> 0.3684210526315789
 |  <PStep:sylib

In [17]:
class BaseClf:
    def auc_score(self, X, y):
        predictions = self.predict(X)
        return roc_auc_score(y, predictions)
    
class IntensityClassifier(BaseClf):
    def predict(self, X):
        predictions = list()
        for s in X:
            predictions.append(get_plurality(s.statement, [ss.statement for ss in s.inputs]))
        
        predictions = np.array(predictions)
        
        #Get max value and normalize by it
        max_pred = predictions.max()
        predictions = predictions / max_pred
        
        #According to the IDV paper, the higher the plurality the better. Since we want
        # class 1 to be the class to be hidden, we have to invert the normalized plurality.
        predictions = 1 - predictions
        return predictions
        
    
class ObviousnessClassifier(BaseClf):
    def predict(self, X):
        try:
            predictions = list()
            for s in X:
                predictions.append(get_theorem_complexity(s.label))

            predictions = np.array(predictions)
            #Here the smallest complexity should receive value of 1
            #so we can signalize it to be hidden

            #Here the biggest complexity becomes 1
            predictions = predictions / predictions.max()
            
            #So them we invert it and put as 0
            predictions = 1 - predictions
        except Exception as e:
            raise e
            
        return predictions
            
class WeightClassifier(BaseClf):   
    def predict(self, X):
        predictions = list()
        for s in X:
            predictions.append(len(s.statement.split(" ")))
        
        predictions = np.array(predictions)
        
        #Get max value and normalize by it
        max_pred = predictions.max()
        predictions = predictions/max_pred
        
        # The way it is the largest statement will get the value of 1
        # Here we will treat the positive class as the statement to be removed
        # In this classifier we want to keep only the small expressions      
        return predictions
    
class ComplexityClassifier(BaseClf):   
    def predict(self, X):
        predictions = list()
        for s in X:
            predictions.append(len(set(s.statement.split(" "))))
        
        predictions = np.array(predictions)
        
        #Get max value and normalize by it
        max_pred = predictions.max()
        predictions = predictions / max_pred
        
        # The way it is the largest statement will get the value of 1
        # Here we will treat the positive class as the statement to be removed
        # In this classifier we want to keep only the small expressions      
        return predictions
    
class TopologyClassifier(BaseClf):
    #This classifier aimed to sample nodes uniformilly accross
    #the tree topology, however it seems that the random classifier already
    #does that fundamentally.
    def predict(self, X):
        pass
    
class EdgesCountClassifier(BaseClf):
    def predict(self, X):
        pass
    
class RandomClassifier(BaseClf):
    def predict(self, X):
        return np.random.random(len(X))

#Single label classifiers doesnt work properly because AUC doesnt work with single classifications
# class ZeroClassifier(BaseClf):
#     def predict(self, X):
#         return np.zeros(len(X))
    
# class OnesClassifier(BaseClf):
#     def predict(self, X):
#         return np.ones(len(X))
    
rand_clf = RandomClassifier()
weight_clf = WeightClassifier()
obvious_clf = ObviousnessClassifier()
comp_clf = ComplexityClassifier()
inten_clf = IntensityClassifier()
#zero_clf = ZeroClassifier()
#ones_clf = OnesClassifier()

In [29]:
def get_messed_prop_auc(prop, score_method=None, n_steps=50, n_expansions=20):
    """
        Generate random expansions to check how the score method can detect additional steps.
    """
    
    auc_values = []
    for _ in range(n_steps):

        prop_proof = construct_proof(database.propositions[prop])
        gt_statements = [s.statement for s in prop_proof.get_steps_df()]

        #Randomly expand steps
        for i in range(n_expansions):
            prop_proof = random.choice(prop_proof.get_steps_df()).expand().get_root_step()

        prop_steps = prop_proof.get_steps_df()
        #Everything within the original statements should not be hidden (class 0)
        prop_ys = [int(s.statement not in gt_statements) for s in prop_steps]
        
        try:
            auc_value = score_method(prop_steps, prop_ys)
        except Exception as e:
            print(prop_ys)
            raise e
        auc_values.append(auc_value)

    return np.mean(auc_values)

In [32]:
%%time 

prop_label = "dfss2"

print(get_messed_prop_auc(prop_label, inten_clf.auc_score, 50, 20))
print(get_messed_prop_auc(prop_label, comp_clf.auc_score, 50, 20))
#print(get_messed_prop_auc(prop_label, obvious_clf.auc_score, 50, 20))
print(get_messed_prop_auc(prop_label, weight_clf.auc_score, 50, 20))
print(get_messed_prop_auc(prop_label, rand_clf.auc_score, 50, 20))

0.21254581204666437
0.4848909851246348
0.7652371053219642
0.5055632698176888
CPU times: user 2.61 s, sys: 54.2 ms, total: 2.67 s
Wall time: 2.67 s


In [117]:
database.propositions["bitr4i"].hyps[5].tree

IndexError: list index out of range

In [103]:
tdb["bitr4i"]["complexity"]

10727

In [33]:
theorem_list = [
    "dfss3",
    "dfss2",
    "dfss",
    "t1lucas",
    "t2lucas",
    "ssun1",
    "t4lucas",
    "t5lucas",
    "opoe",
    "omoe",
    "pwin",
    "inidm",
    "in0",
    "sstr",
    "ssequn1",
    "pwunss",
    #"epee",
    #"emee",
    #"oddp1eveni",
    "dvdsadd2b",
    #"opoeALTV",
    #"omoeALTV"
]

#Ensure everything exists
for t in theorem_list:
    if t not in database.propositions.keys():
        print(f"{t} not in database.")

opoe not in database.
omoe not in database.
dvdsadd2b not in database.


In [34]:
auc_vals = []
for t in tqdm(theorem_list):
    if t in database.propositions.keys():
        auc_val = get_messed_prop_auc(t, inten_clf.auc_score, 50, 20)
        auc_vals.append(auc_val)
    
print(np.mean(auc_vals))

100%|███████████████████████████████████████████████████████████████████████████████████| 17/17 [00:13<00:00,  1.22it/s]

0.2699823328186746


In [35]:
auc_vals = []
for t in tqdm(theorem_list):
    if t in database.propositions.keys():
        auc_val = get_messed_prop_auc(t, comp_clf.auc_score, 50, 20)
        auc_vals.append(auc_val)
    
print(np.mean(auc_vals))

100%|███████████████████████████████████████████████████████████████████████████████████| 17/17 [00:09<00:00,  1.76it/s]

0.7199242636429144


In [36]:
auc_vals = []
for t in tqdm(theorem_list):
    if t in database.propositions.keys():
        auc_val = get_messed_prop_auc(t, obvious_clf.auc_score, 50, 20)
        auc_vals.append(auc_val)
    
print(np.mean(auc_vals))

  0%|                                                                                            | 0/17 [00:01<?, ?it/s]

[0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


AssertionError: Error while getting theorem 'bi1'.

In [37]:
auc_vals = []
for t in tqdm(theorem_list):
    if t in database.propositions.keys():
        auc_val = get_messed_prop_auc(t, weight_clf.auc_score, 50, 20)
        auc_vals.append(auc_val)
    
print(np.mean(auc_vals))

100%|███████████████████████████████████████████████████████████████████████████████████| 17/17 [00:12<00:00,  1.32it/s]

0.7645520140237883


In [43]:
listar teoremas e tirar media das performances
verificar como computar metricas como numero de inferencias necessárias para cada passo


provavelmente vou ter salvar a parte que importa do database em um pickle e dentro disso ir adicionando a quantidade de passos de prova para cada prova 


SyntaxError: invalid syntax (1493545937.py, line 1)

In [78]:


tdb = TheoremDatabase("tdb")

Database not found. Creating new database.


In [77]:
class LightProp:
    def __init__(self, prop):
        pass
    
    

lp = LightProp(database.propositions["dfss2"])

lp_proof = construct_proof(lp)

AttributeError: 'LightProp' object has no attribute 'entails_proof_steps'

In [68]:
len(database.propositions_list)

16080

In [70]:
database.propositions_list[1000].tree

<Tree: wi(wa(wph(),wb(wps(),wph())),wps())>